In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!mkdir -p /tmp/pip/cache/
!cp ../input/segmentationmodelspytorch/segmentation_models/efficientnet_pytorch-0.6.3.xyz /tmp/pip/cache/efficientnet_pytorch-0.6.3.tar.gz
!cp ../input/segmentationmodelspytorch/segmentation_models/pretrainedmodels-0.7.4.xyz /tmp/pip/cache/pretrainedmodels-0.7.4.tar.gz
!cp ../input/segmentationmodelspytorch/segmentation_models/segmentation-models-pytorch-0.1.2.xyz /tmp/pip/cache/segmentation_models_pytorch-0.1.2.tar.gz
!cp ../input/segmentationmodelspytorch/segmentation_models/timm-0.1.20-py3-none-any.whl /tmp/pip/cache/
!cp ../input/segmentationmodelspytorch/segmentation_models/timm-0.2.1-py3-none-any.whl /tmp/pip/cache/
!pip install --no-index --find-links /tmp/pip/cache/ efficientnet-pytorch
!pip install --no-index --find-links /tmp/pip/cache/ segmentation-models-pytorch

In [ ]:
import os
import time
import torch
import shutil
import pandas as pd
from skimage import io, transform
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split

from torchvision import transforms, utils
from torch import nn
import albumentations as A
from albumentations.pytorch import ToTensor
#import tqdm as tqdm
from tqdm import tqdm as tqdm

from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import torch.nn.functional as F
from torch import nn

from albumentations import (HorizontalFlip, ShiftScaleRotate, Normalize, Resize, Compose, GaussNoise)
import cv2
from zipfile import ZipFile

import random

from segmentation_models_pytorch.unet import Unet

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
!mkdir -p /root/.cache/torch/hub/checkpoints/
!cp ../input/efficientnet-pytorch-b0-b7/efficientnet-b0-355c32eb.pth /root/.cache/torch/hub/checkpoints/
!cp ../input/efficientnet-pytorch-b0-b7/efficientnet-b1-f1951068.pth /root/.cache/torch/hub/checkpoints/
!cp ../input/efficientnet-pytorch-b0-b7/efficientnet-b2-8bb594d6.pth /root/.cache/torch/hub/checkpoints/
!cp ../input/efficientnet-pytorch-b0-b7/efficientnet-b3-5fb5a3c3.pth /root/.cache/torch/hub/checkpoints/
!cp ../input/efficientnet-pytorch-b0-b7/efficientnet-b4-6ed6700e.pth /root/.cache/torch/hub/checkpoints/
!cp ../input/efficientnet-pytorch-b0-b7/efficientnet-b5-b6417697.pth /root/.cache/torch/hub/checkpoints/
!cp ../input/efficientnet-pytorch-b0-b7/efficientnet-b6-c76e70fd.pth /root/.cache/torch/hub/checkpoints/
!cp ../input/efficientnet-pytorch-b0-b7/efficientnet-b7-dcc49843.pth /root/.cache/torch/hub/checkpoints/

In [ ]:
# Extracting Train zip file
file_name = "../input/data-science-bowl-2018/stage1_train.zip"
with ZipFile(file_name, 'r') as zip: 
    print('Extracting the files') 
    zip.extractall("stage1_train") 
    print('ok!')

In [ ]:
# Extracting Test zip file
file_name = "../input/data-science-bowl-2018/stage1_test.zip"
with ZipFile(file_name, 'r') as zip: 
    print('Extracting the files') 
    zip.extractall("stage1_test") 
    print('ok!')

In [ ]:
TRAIN_PATH = 'stage1_train/'
TEST_PATH = 'stage1_test/'

In [ ]:
#Load Datasets
#--------------------------#
#Albumentation
def get_train_transform():
    return A.Compose(
       [
        A.Resize(256, 256),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        A.HorizontalFlip(p=0.25),
        A.VerticalFlip(p=0.25),
        ToTensor()
        ])
##Dataset Loader
class LoadDataSet(Dataset):
        def __init__(self,path, transform=None):
            self.path = path
            self.folders = os.listdir(path)
            self.transforms = get_train_transform()
            
        
        def __len__(self):
            return len(self.folders)
              
        
        def __getitem__(self,idx):
            ### link of image
            image_folder = os.path.join(self.path,self.folders[idx],'images/')
            img_path = os.path.join(image_folder,os.listdir(image_folder)[0])
            
            ### read image and resize via transform image 128x128
            img = io.imread(img_path)[:,:,:3].astype('float32')
            img = transform.resize(img,(128,128))
            
            ### link of masks
            if self.path == 'stage1_train/' :
                mask_folder = os.path.join(self.path,self.folders[idx],'masks/')
                ### read mask and resize via transform image 128x128 and return for masks max
                mask = np.zeros((128, 128, 1))
                for mask_ in os.listdir(mask_folder):
                    path_mask_ = os.path.join(mask_folder,mask_)
                    mask_ = io.imread(path_mask_)
                    mask_ = transform.resize(mask_,(128,128))
                    mask_ = np.expand_dims(mask_,axis=-1)
                    mask = np.maximum(mask,mask_)
                mask = mask.astype('float32')

                # agmunted img ans masks via transfomer :get_train_transform()

                agmuented = self.transforms(image=img,mask=mask)
                img = agmuented['image']
                mask = agmuented['mask']
                mask = mask[0].permute(2,0,1)

                return img,mask
             # agmunted img ans masks via transfomer :get_train_transform()

            agmuented = self.transforms(image=img)
            img = agmuented['image']
             
                
            return img
               
            
            



In [ ]:
train_dataset = LoadDataSet(TRAIN_PATH, transform=get_train_transform())
image , mask = train_dataset.__getitem__(4)

In [ ]:
print('train',image.shape)
print('train',mask.shape)

In [ ]:
test_dataset = LoadDataSet(TEST_PATH, transform=get_train_transform())
image_test = test_dataset.__getitem__(4)

In [ ]:
print('test',image_test.shape)


In [ ]:
train_dataset.__len__()

In [ ]:
test_dataset.__len__()

In [ ]:
image_test

In [ ]:
image

In [ ]:
mask

In [ ]:
def imshow(img,mask):
    img = img / 2 + 0.5     # unnormalize
    mask = mask / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(npimg.T)
    plt.show()
    plt.imshow(mask.T, interpolation="nearest")
    plt.show()

In [ ]:
imshow(image,mask)

In [ ]:
def format_image(img):
    img = np.array(np.transpose(img, (1,2,0)))
    mean=np.array((0.485, 0.456, 0.406))
    std=np.array((0.229, 0.224, 0.225))
    img  = std * img + mean
    img = img*255
    img = img.astype(np.uint8)
    return img
def format_mask(mask):
    mask = np.squeeze(np.transpose(mask, (1,2,0)))
    return mask

In [ ]:
format_image()

In [ ]:
# Visualize images as well as masks
def visualize_dataset(n_images, predict=None):
    images = random.sample(range(0, 670), n_images)
    figure, ax = plt.subplots(nrows=len(images), ncols=2, figsize=(5, 8))

    for i in range(0, len(images)):
        img_no = images[i]
        print(images[i])
        image, mask = train_dataset.__getitem__(img_no)
        image = format_image(image)
        mask = format_mask(mask)
        ax[i, 0].imshow(image)
        ax[i, 1].imshow(mask, interpolation="nearest")
        ax[i, 0].set_title("Ground Truth Image"+str(i))
        ax[i, 1].set_title("Mask")
        ax[i, 0].set_axis_off()
        ax[i, 1].set_axis_off()
    
    plt.tight_layout()
    plt.show()

In [ ]:
visualize_dataset(3)

In [ ]:
# Visualize images as well as masks
def visualize_dataset_test(n_images, predict=None):
    images = random.sample(range(0, 65), n_images)
    figure, ax = plt.subplots(nrows=len(images), ncols=2, figsize=(5, 8))

    for i in range(0, len(images)):
        img_no = images[i]
        image = test_dataset.__getitem__(img_no)
        image = format_image(image)
#         mask = format_mask(mask)
        ax[i, 0].imshow(image)
#         ax[i, 1].imshow(mask, interpolation="nearest")
        ax[i, 0].set_title("Ground Truth Image"+str(i))
#         ax[i, 1].set_title("Mask")
        ax[i, 0].set_axis_off()
#         ax[i, 1].set_axis_off()
    
    plt.tight_layout()
    plt.show()

In [ ]:
visualize_dataset_test(4)

In [ ]:
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split
def train_val_dataset(dataset,val_split=0.25):
    train_idx, val_idx = train_test_split(list(range(len(train_dataset))), test_size=val_split)
    datasets = {}
    datasets['train'] = Subset(train_dataset, train_idx)
    datasets['val'] = Subset(train_dataset, val_idx)
    return datasets

In [ ]:
print(len(train_dataset))
datasets = train_val_dataset(train_dataset)
train_data = datasets['train']
valid_data = datasets['val']
print("Length of train and valid datas: {}, {}".format(len(train_data), len(valid_data)))

In [ ]:
train_loader = DataLoader(dataset=train_data, batch_size=10, shuffle=True)

val_loader = DataLoader(dataset=valid_data, batch_size=10)
test_loader = DataLoader(dataset=test_dataset, batch_size=10,shuffle=True)

In [ ]:
DATA , TARGET = next(iter(train_loader))

In [ ]:
DATA[4]

In [ ]:
TARGET[4]

In [ ]:
import torchvision
imshow(torchvision.utils.make_grid(DATA[4]),torchvision.utils.make_grid(TARGET[4]))

In [ ]:
##########  Model #######

In [ ]:
class CLUTTERDCELLS(nn.Module):
    def __init__(self):
        super(CLUTTERDCELLS, self).__init__()
        self.cnn_model = Unet('efficientnet-b6', encoder_weights="imagenet", classes=1, activation=None)
        #self.cnn_model.decoder.blocks.append(self.cnn_model.decoder.blocks[-1])
        #self.cnn_model.decoder.blocks[-2] = self.cnn_model.decoder.blocks[-3]
    
    def forward(self, imgs):
        img_segs = self.cnn_model(imgs)
        
        return img_segs

In [ ]:
model = CLUTTERDCELLS().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr = 1e-3)

In [ ]:
model(DATA.to(device))

In [ ]:
print(DATA.shape)

In [ ]:
####### LOsss #######


In [ ]:
class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        Dice_BCE = BCE + dice_loss
        
        return Dice_BCE


class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        #inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        
        return 1 - dice


class IoU(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(IoU, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        #intersection is equivalent to True Positive count
        #union is the mutually inclusive area of all labels & predictions 
        intersection = (inputs * targets).sum()
        total = (inputs + targets).sum()
        union = total - intersection 
        
        IoU = (intersection + smooth)/(union + smooth)
                
        return IoU

In [ ]:
def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

In [ ]:
if not os.path.exists("model"):
    os.makedirs("model")

In [ ]:
#####" train ########"

In [ ]:
checkpoint_path = 'model/chkpoint_'
best_model_path = 'model/bestmodel.pt'

In [ ]:
#from engine import evaluate
criterion = DiceBCELoss()
accuracy_metric = IoU()
num_epochs=20
valid_loss_min = np.Inf

total_train_loss = []
total_train_score = []
total_valid_loss = []
total_valid_score = []
losses_value = 0
for epoch in range(num_epochs):
  
    train_loss = []
    train_score = []
    valid_loss = []
    valid_score = []
 #<-----------Training Loop---------------------------->
    pbar = tqdm(train_loader, desc = 'description')
    for x_train, y_train in pbar:
        x_train = torch.autograd.Variable(x_train).to(device)
        y_train = torch.autograd.Variable(y_train).to(device)
        optimizer.zero_grad()
        output = model(x_train).to(device)
      #Loss
        loss = criterion(output, y_train)
        losses_value = loss.item()
      #Score
        score = accuracy_metric(output,y_train)
        loss.backward()
        optimizer.step()
        train_loss.append(losses_value)
        train_score.append(score.item())
      #train_score.append(score)
        pbar.set_description(f"Epoch: {epoch+1}, loss: {losses_value:.4f}, IoU: {score:4f}")
#     total_train_loss.append(np.mean(train_loss))
#     total_train_score.append(np.mean(train_score))
#     print(f"\n###############Train Loss: {total_train_loss[-1]:.4f}, Train IOU: {total_train_score[-1]:.4f}###############")

    #<---------------Validation Loop---------------------->
    with torch.no_grad():
        for image,mask in val_loader:
            image = torch.autograd.Variable(image).cuda()
            mask = torch.autograd.Variable(mask).cuda()
            output = model(image)
            ## Compute Loss Value.
            loss = criterion(output, mask)
            losses_value = loss.item()
            ## Compute Accuracy Score
            score = accuracy_metric(output,mask)
            valid_loss.append(losses_value)
            valid_score.append(score.item())

    total_train_loss.append(np.mean(train_loss))
    total_train_score.append(np.mean(train_score))
    total_valid_loss.append(np.mean(valid_loss))
    total_valid_score.append(np.mean(valid_score))
    print(f"\n###############Train Loss: {total_train_loss[-1]:.4f}, Train IOU: {total_train_score[-1]:.4f}###############")
    print(f"###############Valid Loss: {total_valid_loss[-1]:.4f}, Valid IOU: {total_valid_score[-1]:.4f}###############")
    
    #Save best model Checkpoint
    # create checkpoint variable and add important data
    checkpoint = {
        'epoch': epoch + 1,
        'valid_loss_min': total_valid_loss[-1],
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
    }
    
    # save checkpoint
    save_ckp(checkpoint, False, checkpoint_path, best_model_path)
    
    ## TODO: save the model if validation loss has decreased
    if total_valid_loss[-1] <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,total_valid_loss[-1]))
        # save checkpoint as best model
        save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        valid_loss_min = total_valid_loss[-1]

    

In [ ]:
import seaborn as sns

plt.figure(1)
plt.figure(figsize=(15,5))
sns.set_style(style="darkgrid")
plt.subplot(1, 2, 1)
sns.lineplot(x=range(1,num_epochs+1), y=total_train_loss, label="Train Loss")
sns.lineplot(x=range(1,num_epochs+1), y=total_valid_loss, label="Valid Loss")
plt.title("Loss")
plt.xlabel("epochs")
plt.ylabel("DiceBCELoss")

plt.subplot(1, 2, 2)
sns.lineplot(x=range(1,num_epochs+1), y=total_train_score, label="Train Score")
sns.lineplot(x=range(1,num_epochs+1), y=total_valid_score, label="Valid Score")
plt.title("Score (IoU)")
plt.xlabel("epochs")
plt.ylabel("IoU")
plt.show()

In [ ]:
#loading the saved model
model, optimizer, start_epoch, valid_loss_min = load_ckp(checkpoint_path, model, optimizer)

In [ ]:
def visualize_predict(model, n_images):
  #model = model.eval()
  figure, ax = plt.subplots(nrows=n_images, ncols=3, figsize=(15, 18))
  with torch.no_grad():
    for data,mask in val_loader:
        data = torch.autograd.Variable(data, volatile=True).cuda()
        mask = torch.autograd.Variable(mask, volatile=True).cuda()
        o = model(data)
        o= F.sigmoid(o)
        break
  for img_no in range(0, n_images):
    tm=o[img_no][0].data.cpu().numpy()
    img = data[img_no].data.cpu()
    msk = mask[img_no].data.cpu()
    img = format_image(img)
    msk = format_mask(msk)
    ax[img_no, 0].imshow(img)
    ax[img_no, 1].imshow(msk, interpolation="nearest")
    ax[img_no, 2].imshow(tm, interpolation="nearest")
    ax[img_no, 0].set_title(" Image")
    ax[img_no, 1].set_title(" Mask Image")
    ax[img_no, 2].set_title("Predicted Mask")
    ax[img_no, 0].set_axis_off()
    ax[img_no, 1].set_axis_off()
    ax[img_no, 2].set_axis_off()
  plt.tight_layout()
  plt.show()

In [ ]:
visualize_predict(model, 4)

In [ ]:
def visualize_predict_test(model, n_images):
  #model = model.eval()
  figure, ax = plt.subplots(nrows=n_images, ncols=2, figsize=(15, 18))
  with torch.no_grad():
    for data in test_loader:
        data = torch.autograd.Variable(data, volatile=True).cuda()
#         mask = torch.autograd.Variable(mask, volatile=True).cuda()
        o = model(data)
        o= F.sigmoid(o)
        break
  for img_no in range(0, n_images):
    tm=o[img_no][0].data.cpu().numpy()
    img = data[img_no].data.cpu()
#     msk = mask[img_no].data.cpu()
    img = format_image(img)
#     msk = format_mask(msk)
    ax[img_no, 0].imshow(img)
#     ax[img_no, 1].imshow(msk, interpolation="nearest")
    ax[img_no, 1].imshow(tm, interpolation="nearest")
    ax[img_no, 0].set_title(" Image")
#     ax[img_no, 1].set_title(" Mask Image")
    ax[img_no, 1].set_title("Predicted Mask")
    ax[img_no, 0].set_axis_off()
    ax[img_no, 1].set_axis_off()
#     ax[img_no, 2].set_axis_off()
  plt.tight_layout()
  plt.show()

In [ ]:
visualize_predict_test(model,10)